# TO-DO
- Refactor the names/ add comments to better understand. 
- Set predefined Transforms, for both Training (ToTensor + Standardize +  Data Augmentation)
- Set predefined Transforms, Validation & Test (ToTensor + Standardize) 
- Add the possibility to save weights and restart the training from them.
- implement attention map.
- increase n_heads,n_blocks
- try logSoftmax
- Implement Halting Thing: assign to each token an halting score $h_{k}^{l}$ where k= k-th token and l = layers depth  

https://arxiv.org/pdf/2112.07658.pdf
https://medium.com/mlearning-ai/vision-transformers-from-scratch-pytorch-a-step-by-step-guide-96c3313c2e0c

### 0: IMPORTING LIBRARIES AND SETTING THE SEEDS

In [ ]:
# Importing necessary libraries
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchmetrics.classification import Accuracy, MulticlassF1Score, MulticlassPrecision, MulticlassRecall

from pathlib import Path
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import TQDMProgressBar, LearningRateMonitor, ModelCheckpoint
from torch.optim.lr_scheduler import StepLR

from typing import Tuple
import PIL
import random
from PIL import Image
import csv
import numpy as np
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import classification_report
import seaborn as sns
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.utils.class_weight import compute_class_weight
import math
import os

In [ ]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results.
    
    Arguments:
        - seed {int} : Number of the seed.
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    pl.seed_everything(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed.
seed_everything(31)

### 1: DATA INSPECTION

#### 1.1: CREATION OF THE LABEL DICTIONARY

In [ ]:
mapping_dict = {}

# Open the file in read mode.
with open('/kaggle/input/tiny-imagenet/tiny-imagenet-200/words.txt', 'r') as file:
    
    # Read each line from the file.
    for line in file:
        # Split the line into tokens based on whitespace.
        tokens = line.strip().split('\t')
        
        # Check if there are at least two tokens.
        if len(tokens) >= 2:
            # Extract the encoded label (left) and actual label (right).
            encoded_label, actual_label = tokens[0], tokens[1]
            
            # Add the mapping to the dictionary.
            mapping_dict[encoded_label] = actual_label

# Print the mapping dictionary.
#print(mapping_dict)


#### 1.2: DISPLAYING EXAMPLES OF THE DATASET

In [ ]:
# Loading the dataset using ImageFolder.
dataset0 = datasets.ImageFolder(root="/kaggle/input/tiny-imagenet/tiny-imagenet-200/train/", transform=None)

# Extract class names and their counts.
class_names = dataset0.classes
class_counts = [dataset0.targets.count(i) for i in range(len(class_names))]
np.random.seed(31)

# Create a grid of 10 images with labels.
plt.figure(figsize=(15, 8))
for i in range(10):
    
    # Randomly select an image and its corresponding label.
    index = np.random.randint(len(dataset0))
    image, label = dataset0[index]

    # Display the image with its label
    plt.subplot(2, 5, i+1)
    plt.imshow(np.array(image))  # Convert the PIL Image to a numpy array
    plt.title(f"Label: {class_names[label]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

#### 1.3: DISPLAYING EXAMPLES OF THE DATASET WITH DECODED LABELS

In [ ]:
# Assuming you have already loaded the dataset using ImageFolder
dataset0 = datasets.ImageFolder(root="/kaggle/input/tiny-imagenet/tiny-imagenet-200/train/", transform=None)

# Extract class names and their counts
class_names = dataset0.classes
class_counts = [dataset0.targets.count(i) for i in range(len(class_names))]

np.random.seed(31)
# Create a grid of 10 images with labels
plt.figure(figsize=(15, 8))
for i in range(10):
    # Randomly select an image and its corresponding label
    index = np.random.randint(len(dataset0))
    image, encoded_label = dataset0[index]
    # Look up the actual label using the mapping dictionary
    actual_label = mapping_dict.get(class_names[encoded_label], "Unknown Label")
    
    # Trim the label if it exceeds the maximum length.
    actual_label_trimmed = actual_label[:15] + '...' if len(actual_label) > 15 else actual_label

    # Display the image with its label.
    plt.subplot(2, 5, i+1)
    plt.imshow(np.array(image))  # Convert the PIL Image to a numpy array
    plt.title(f"Label: {actual_label_trimmed}", wrap=True)
    plt.axis('off')

plt.tight_layout()
plt.show()

### 2: DATA-MODULE DEFINITION

#### 2.0: CUSTOMIZED TRANSFORM CLASS

In [ ]:
class AdaViT_Transformations:
    
    def __init__(self):
        pass
        #self.image_size = image_size

    def __call__(self, sample):
        
        # Implementation of a series of Customized Transformations.
        transform = transforms.Compose([
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        return transform(sample)

#### 2.1: CUSTOMIZED TRAINING SET VERSION

In [ ]:
class CustomTrainingTinyImagenet(ImageFolder):
    
    def __init__(self, root, transform=None):
        super(CustomTrainingTinyImagenet, self).__init__(root, transform=transform)
        self.class_to_index = {cls: idx for idx, cls in enumerate(sorted(self.classes))}
        self.index_to_class = {idx: cls for cls, idx in self.class_to_index.items()}

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)

        # Adjust the directory depth
        target_str = os.path.basename(os.path.dirname(os.path.dirname(path)))  # Remove two os.path.dirname

        # Convert string label to numerical index
        target = self.class_to_index[target_str]

        return sample, target

    def get_class_from_index(self, index):
        return self.index_to_class[index]


#### 2.2: CUSTOMIZED VALIDATION SET VERSION

In [ ]:
class CustomValidationTinyImagenet(pl.LightningDataModule):
    
    def __init__(self, root, transform=None):
        self.root = Path(root)
        self.transform = transform
        self.labels = self.load_labels()
        self.label_to_index = {label: idx for idx, label in enumerate(sorted(set(self.labels.values())))}
        self.index_to_label = {idx: label for label, idx in self.label_to_index.items()}

    def load_labels(self):
        label_path = "/kaggle/input/tiny-imagenet/tiny-imagenet-200/val/val_annotations.txt"
        labels = {}

        with open(label_path, "r") as f:
            lines = f.readlines()

        for line in lines:
            parts = line.split("\t")
            image_name, label = parts[0], parts[1]
            labels[image_name] = label

        return labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        image_name = f"val_{index}.JPEG"
        image_path = self.root / image_name

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        # Use get method to handle cases where the key is not present
        label_str = self.labels.get(image_name, 'Label not found')

        # Convert string label to numerical index
        label = self.label_to_index[label_str]

        return image, label

    def get_label_from_index(self, index):
        return self.index_to_label[index]


#### 2.3: GENERAL DATA-MODULE DEFINITION

In [ ]:
class AViT_DataModule(pl.LightningDataModule):
    
    def __init__(self, train_data_dir, val_data_dir, batch_size, num_workers=4):
        super(AViT_DataModule, self).__init__()
        self.train_data_dir = train_data_dir
        self.val_data_dir = val_data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        
        self.transform = AdaViT_Transformations()

    def setup(self, stage=None):
        # Load Train dataset using CustomTrainingTinyImagenet with the new directory structure
        self.train_dataset = CustomTrainingTinyImagenet(self.train_data_dir, transform=self.transform)

        # Load Validation dataset
        self.val_dataset = CustomValidationTinyImagenet(self.val_data_dir, transform=self.transform)

    def train_dataloader(self):
        # Return the DataLoader for the training dataset
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True)

    def val_dataloader(self):
        # Return the DataLoader for the validation dataset
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)


#### 2.4: TESTING TRAINING AND VALIDATION DATALOADERS

In [ ]:
# Define the AViT_DataModule
data_module = AViT_DataModule(
    train_data_dir="/kaggle/input/tiny-imagenet/tiny-imagenet-200/train/",
    val_data_dir="/kaggle/input/tiny-imagenet/tiny-imagenet-200/val/images/",
    batch_size=512  # Adjust the batch size as needed
)

# Setup the dataloaders
data_module.setup()

# Get a batch from the training DataLoader
train_dataloader = data_module.train_dataloader()
train_batch = next(iter(train_dataloader))

# Get a batch from the validation DataLoader
val_dataloader = data_module.val_dataloader()
val_batch = next(iter(val_dataloader))

# Display images and labels
def show_images_labels(images, labels, title):
    fig, axs = plt.subplots(1, len(images), figsize=(8, 4))
    fig.suptitle(title)

    for i, (img, label) in enumerate(zip(images, labels)):
        axs[i].imshow(transforms.ToPILImage()(img))
        axs[i].set_title(f"Label: {label}")
        axs[i].axis('off')
    
    plt.show()

# Show two images from the training batch
show_images_labels(train_batch[0][:2], train_batch[1][:2], title='Training Batch')

# Show two images from the validation batch
show_images_labels(val_batch[0][:2], val_batch[1][:2], title='Validation Batch')

### 3: MODEL DEFINITION

#### 3.0: PATCHING FUNCTION DEFINITION

In [ ]:
def Make_Patches_from_Image(images, n_patches):
    n, c, h, w = images.shape

    assert h == w, "Make_Patches_from_Image method is implemented for square images only"

    patches = torch.zeros(n, n_patches ** 2, h * w * c // n_patches ** 2)
    patch_size = h // n_patches

    for idx, image in enumerate(images):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = image[:, i * patch_size: (i + 1) * patch_size, j * patch_size: (j + 1) * patch_size]
                patches[idx, i * n_patches + j] = patch.flatten()
    return patches

In [ ]:
# Helper function to visualize patches
def visualize_patches(images, n_patches, title):
    patches = Make_Patches_from_Image(images, n_patches)
    
    fig, axs = plt.subplots(n_patches, n_patches, figsize=(8, 8))
    fig.suptitle(title)
    
    patch_size = images.shape[-1] // n_patches

    for i in range(n_patches):
        for j in range(n_patches):
            patch_index = i * n_patches + j
            # Reshape each patch to (3, patch_size, patch_size)
            patch = patches[0, patch_index].reshape(3, patch_size, patch_size).cpu().numpy()
            axs[i, j].imshow(patch.transpose(1, 2, 0))
            axs[i, j].axis('off')

    plt.show()

# Visualize patches for a training image
visualize_patches(train_batch[0], n_patches=4, title='Training Patches')

# Visualize patches for a validation image
visualize_patches(val_batch[0], n_patches=4, title='Validation Patches')


#### 3.1: POSITIONAL EMBEDDING DEFINITION

In [ ]:
def get_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result

#### 3.2: MULTI-HEAD SELF-ATTENTION DEFINITION

In [ ]:
class MyMSA(nn.Module):
    def __init__(self, d, n_heads=2):
        super(MyMSA, self).__init__()
        self.d = d
        self.n_heads = n_heads

        assert d % n_heads == 0, f"Can't divide dimension {d} into {n_heads} heads"

        d_head = int(d / n_heads)
        self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.d_head = d_head
        self.softmax = nn.Softmax(dim=-1)
        
        # Initialize weights.
        self.initialize_weights_msa()
        
    def forward(self, sequences):
        # Sequences has shape (N, seq_length, token_dim)
        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)
        # And come back to    (N, seq_length, item_dim)  (through concatenation)
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]

                seq = sequence[:, head * self.d_head: (head + 1) * self.d_head]
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)

                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])
    
    def initialize_weights_msa(self):
        # Initialize weights for linear layers
        for q_mapping, k_mapping, v_mapping in zip(self.q_mappings, self.k_mappings, self.v_mappings):
            nn.init.xavier_uniform_(q_mapping.weight)
            nn.init.xavier_uniform_(k_mapping.weight)
            nn.init.xavier_uniform_(v_mapping.weight)

#### 3.3: ViT BLOCK DEFINITION

In [ ]:
class MyViTBlock(nn.Module):
    def __init__(self, hidden_d, n_heads, mlp_ratio=10):
        super(MyViTBlock, self).__init__()
        
        self.hidden_d = hidden_d
        self.n_heads = n_heads

        self.norm1 = nn.LayerNorm(hidden_d)
        self.mhsa = MyMSA(hidden_d, n_heads)
        self.norm2 = nn.LayerNorm(hidden_d)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_d, mlp_ratio * hidden_d),
            nn.GELU(),
            nn.Linear(mlp_ratio * hidden_d, hidden_d)
        )
        
        # Initialize weights.
        self.initialize_weights_block()

    def forward(self, x):
        
        out = x + self.mhsa(self.norm1(x))
        out = out + self.mlp(self.norm2(out))
        return out
    
    def initialize_weights_block(self):
        
        # Initialize weights for linear layers in mlp.
        for layer in self.mlp:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)

#### 3.4: ViT MODEL DEFINITION

In [ ]:
class MyViT(nn.Module):
        
    def __init__(self, chw, n_patches, n_blocks, hidden_d, n_heads, out_d):
        # Super constructor
        super(MyViT, self).__init__()
        
        # Attributes
        self.chw = chw # ( C , H , W )
        self.n_patches = n_patches
        self.n_blocks = n_blocks
        self.n_heads = n_heads
        self.hidden_d = hidden_d
        self.mlp_ratio=100
        
        # Input and patches sizes
        assert chw[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        assert chw[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        self.patch_size = (chw[1] / n_patches, chw[2] / n_patches)

        # 1) Linear mapper
        self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
        self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)
        
        # 2) Learnable classification token
        self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))
        
        # 3) Positional embedding
        self.register_buffer('positional_embeddings', get_positional_embeddings(n_patches ** 2 + 1, hidden_d), persistent=False)
        
        # 4) Transformer encoder blocks
        self.blocks = nn.ModuleList([MyViTBlock(hidden_d, n_heads) for _ in range(n_blocks)])
        
        # 5) Classification MLPk
        self.mlp = nn.Sequential(
            nn.Linear(self.hidden_d, self.mlp_ratio * self.hidden_d),
            nn.GELU(),
            nn.Linear(self.mlp_ratio * self.hidden_d, out_d),
            nn.LogSoftmax(dim=-1)
        )
        
        # Initialize weights
        self.initialize_weights()

    def forward(self, images):
        # Dividing images into patches
        n, c, h, w = images.shape
        patches = Make_Patches_from_Image(images, self.n_patches).to(self.positional_embeddings.device)
        
        # Running linear layer tokenization
        # Map the vector corresponding to each patch to the hidden size dimension
        tokens = self.linear_mapper(patches)
        
        # Adding classification token to the tokens
        tokens = torch.cat((self.class_token.expand(n, 1, -1), tokens), dim=1)
        
        # Adding positional embedding
        out = tokens + self.positional_embeddings.repeat(n, 1, 1)
        
        # Transformer Blocks
        for block in self.blocks:
            out = block(out)
            
        # Getting the classification token only
        out = out[:, 0]
        
        return self.mlp(out) # Map to output dimension, output category distribution
    

    def initialize_weights(self):
        
        # Initialize weights for linear layers, embeddings, etc.
        nn.init.xavier_uniform_(self.linear_mapper.weight)
        nn.init.normal_(self.class_token.data)

        # Initialize weights for classification MLP
        nn.init.xavier_uniform_(self.mlp[0].weight)
    

#### 3.5: GENERAL AViT MODEL DEFINITION

In [ ]:
class AViT_Model(MyViT, pl.LightningModule):
    
    def __init__(self, input_d, n_patches, n_blocks, hidden_d, n_heads, out_d):
        
        super(AViT_Model, self).__init__(input_d, n_patches, n_blocks, hidden_d, n_heads, out_d)

        # Definition of the Weighted Cross Entropy Loss + Label Smoothing.
        self.loss = CrossEntropyLoss()

        # Definition of Accuracy, F1Score, Precision and Recall Metrics.
        self.acc = Accuracy(task="multiclass", num_classes=out_d)
        self.f1score = MulticlassF1Score(num_classes=out_d, average='macro')
        self.precision = MulticlassPrecision(num_classes=out_d, average='macro')
        self.recall = MulticlassRecall(num_classes=out_d, average='macro')
        
        # Definition of lists to be used in the "on_ ... _epoch_end" functions.
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.test_step_outputs = []
    
    def _step(self, batch):
        
        # Common computation of the metrics among Training, Validation and Test Set.
        x, y = batch
        preds = self(x)
        loss = self.loss(preds, y)
        acc = self.acc(preds, y)
        f1score = self.f1score(preds, y)
        precision = self.precision(preds, y)
        recall = self.recall(preds, y)
        
        return loss, acc, f1score, precision, recall
    
    
    def training_step(self, batch, batch_idx):
        
        # Compute the Training Loss and Accracy.
        loss, acc, _, _, _ = self._step(batch)

        # Create a dictionary to represent the output of the training step.
        training_step_output = {
            "train_loss": loss.item(),
            "train_acc": acc.item()
        }

        # Append the dictionary to the list.
        self.training_step_outputs.append(training_step_output)

        # Perform logging.
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        
        # Compute the Validation Loss and Accuracy.
        loss, acc, _, _, _ = self._step(batch)

        # Create a dictionary to represent the output of the validation step.
        validation_step_output = {
            "val_loss": loss.item(),
            "val_acc": acc.item()
        }

        # Append the dictionary to the list.
        self.validation_step_outputs.append(validation_step_output)

        # Perform logging.
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True, logger=True)

    def on_validation_epoch_end(self):
        
        # Calculate the Mean Loss and Accuracy from the list of dictionaries.
        loss_tot = torch.tensor([item["val_loss"] for item in self.validation_step_outputs]).mean()
        acc_tot = torch.tensor([item["val_acc"] for item in self.validation_step_outputs]).mean()

        # Log the mean values.
        self.log("val_loss", loss_tot)
        self.log("val_acc", acc_tot)

        # Print messages.
        message_loss = f'Epoch {self.current_epoch} Validation Loss -> {loss_tot}'
        message_accuracy = f'      Validation Accuracy -> {acc_tot}'
        print(message_loss + message_accuracy)

        # Clear the list to free memory.
        self.validation_step_outputs.clear()

    
    def configure_optimizers(self):
        
        # Configure the Adam Optimizer.
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        #scheduler = StepLR(optimizer, step_size=10, gamma=0.5)  # Adjust parameters as needed
        #return {"optimizer": optimizer, "lr_scheduler": scheduler}
        
        return optimizer
        

### 4: MODEL TRAINING

#### 4.1: CALLBACKS DEFINITION

In [ ]:
# Checkpoint CallBack Definition.
my_checkpoint_call = ModelCheckpoint(
    dirpath="/kaggle/working/checkpoints/",
    filename="Best_Model",
    monitor="val_loss",
    mode="min",
    save_top_k=1
)

# Learning Rate CallBack Definition.
my_lr_monitor_call = LearningRateMonitor(logging_interval="epoch")

# Early Stopping CallBack Definition.
my_early_stopping_call = pl.callbacks.EarlyStopping(monitor="val_loss", patience=30, mode="min", min_delta=0.001)

# Progress Bar CallBack Definition.
my_progress_bar_call = TQDMProgressBar(refresh_rate=10)

# TensorBoardLogger CallBack Definition.
tb_logger = TensorBoardLogger("/kaggle/working/logs", name="AViT")

# CSV CallBack Definition.
csv_logger = CSVLogger("/kaggle/working/logs", name="AViT")


#### 4.2: MODEL INSTANTIATION & TRAINING

In [ ]:
# Initialize the model and data module.

# Image Flattened Dimension = 64*64*3 = 4096*3 = 12288
# n_patches = 16
# chw = (3, 64, 64)
# patch_size = (3, 64/16, 64/16) = (3, 4, 4) = (#channels, #h_patch, #w_patch)
# 256 patch of 48 elements = 256 * 48 = 12288
# hidden_d = linear mapping output dimension

# n_block = how many times the Transformer Block is repeated. 
# n_heads = how many times we repeat self-attention.
# out_d = number of output classes.

model = AViT_Model((3, 64, 64), 
                   n_patches=8, 
                   n_blocks=4, 
                   hidden_d=32, 
                   n_heads=4, 
                   out_d = 200)

datamodule = AViT_DataModule(train_data_dir="/kaggle/input/tiny-imagenet/tiny-imagenet-200/train/", 
                             val_data_dir="/kaggle/input/tiny-imagenet/tiny-imagenet-200/val/images/", 
                             batch_size=512)

# Setup the dataloaders
data_module.setup()

# Create a PyTorch Lightning Trainer.
trainer = pl.Trainer(
    max_epochs=25,
    accelerator="auto", 
    devices="auto",
    log_every_n_steps=1,
    logger=tb_logger,
    callbacks=[my_progress_bar_call,
               my_checkpoint_call,
               my_lr_monitor_call,
               my_early_stopping_call,
               ]
)


In [ ]:
# Train the model
trainer.fit(model, datamodule)

In [ ]:
#print(result)

### 5: BEST MODEL EXTRAPOLATION

In [ ]:
best_model_path = my_checkpoint_call.best_model_path
best_model = AViT_Model.load_from_checkpoint(checkpoint_path=best_model_path,
                                             input_d=(3, 64, 64), 
                                             n_patches=8, 
                                             n_blocks=4, 
                                             hidden_d=32,
                                             n_heads=4, 
                                             out_d = 200)

### 6: SAVING THE BEST MODEL

In [ ]:
# Save it as a pth file.
# Specify the path where you want to save the model.
model_path = f"/kaggle/working/best_model_acc.pth"

# Save the model's state dict to the specified file.
torch.save(best_model.state_dict(), model_path)

# Save it as a CheckPoint (Specific of PyTorch Lightning = Model State Dictionary + Training State + Optimizer State)
# Specify the path where you want to save the model checkpoint.
ckpt_path = f"/kaggle/working/best_model_acc.ckpt"

# Save the model's state dict to the specified file
torch.save(best_model.state_dict(), ckpt_path)

### 7: TRAINING FROM A SAVED CHECKPOINT

# Specify the path to the checkpoint file
checkpoint_path = "/kaggle/working/best_model_acc_0.001.ckpt"

# Create an instance of your Lightning model (AViT_Model in this case)
loaded_model = AViT_Model.load_from_checkpoint(checkpoint_path=checkpoint_path)

datamodule = AViT_DataModule(train_data_dir="/kaggle/input/tiny-imagenet/tiny-imagenet-200/train/", 
                             val_data_dir="/kaggle/input/tiny-imagenet/tiny-imagenet-200/val/images/", 
                             batch_size=512)

# Specify other Trainer configurations
trainer = Trainer(
    max_epochs=5,
    accelerator="auto", 
    devices="auto",
    log_every_n_steps=1,
    logger=tb_logger,
    callbacks=[my_progress_bar_call,
               my_checkpoint_call,
               my_lr_monitor_call,
               my_early_stopping_call,
               ]
    
    resume_from_checkpoint=checkpoint_path 
)

# Start training
result = trainer.fit(loaded_model, datamodule)


best_model_path = my_checkpoint_call.best_model_path
best_model = AViT_Model.load_from_checkpoint(checkpoint_path=best_model_path, 
                                             input_d=(3, 64, 64), 
                                             n_patches=4, 
                                             n_blocks=1, 
                                             hidden_d=1, 
                                             n_heads=1, 
                                             out_d = 200)

# Save it as a pth file.
# Specify the path where you want to save the model.
model_path = f"/kaggle/working/best_model_acc_{result[0]['val_acc']:.5f}.pth"

# Save the model's state dict to the specified file.
torch.save(best_model.state_dict(), model_path)

# Save it as a CheckPoint (Specific of PyTorch Lightning = Model State Dictionary + Training State + Optimizer State)
# Specify the path where you want to save the model checkpoint.
ckpt_path = f"/kaggle/working/best_model_acc_{result[0]['val_acc']:.5f}.ckpt"

# Save the model's state dict to the specified file
torch.save(best_model.state_dict(), ckpt_path)